In [100]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import os
os.getcwd()

'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/pulling_data'

In [3]:
import wrds

In [67]:
import pandas as pd
import io

In [36]:
db = wrds.Connection(wrds_username='aclime2')
#db.create_pgpass_file()#aclime2
#db.close()

Loading library list...
Done


# OptionMetrics Pull

In [118]:
import sys
current_dir=r'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/pulling_data'
new_dir=r'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/VIX_SKEW_replication'
os.chdir(new_dir)
from VIX_SKEW_replication.box_client import upload_to_box
os.chdir(current_dir)

from OptionMetrics_query import option_metric_query
query=option_metric_query('SPX',2023)
#query=option_metric_query('SPX',2023,month=7,day=3)
print(query)

years=range(2021,2022+1)
ticks=['SPX']


#for tick in ticks:
#    for year in years:
#        print(tick,year)
#        query=option_metric_query(tick,2023)
#        df = db.raw_sql(query)
#        dest=fr'OptionMetrics/{tick}_{year}.csv'
#        dest=fr'OptionMetrics/{tick}_{year}.csv'
#        df.to_csv(dest)

file_paths=[]

for tick in ticks:
    for year in years:
        for month in range(1,12+1)[:]:
            print(tick,year,month)
            #query=option_metric_query(tick,2023,month=month) #bug
            query=option_metric_query(tick,year,month=month)
            try:
                df = db.raw_sql(query)
                dest=fr'OptionMetrics/{tick}_{year}_{month}.csv'
                #df.to_csv(dest)
                csv_data=df.to_csv(dest)
                flo = io.StringIO(csv_data)
                #upload_to_box(flo,dest.split(r'/')[-1])
                upload_to_box(dest,dest.split(r'/')[-1])

                file_paths.append(dest)
                
            except:
                print('skipping')

        


    SELECT *
    FROM optionm.opprcd2023 AS tbl 
    WHERE secid = 108105
    
SPX 2021 1


Request "PUT https://upload.app.box.com/api/2.0/files/upload_sessions/4A43699F61115DF102286C6B0EEFDA41" failed with ConnectionError exception: ConnectionError(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


SPX 2021 2
SPX 2021 3
SPX 2021 4
SPX 2021 5
SPX 2021 6
SPX 2021 7
SPX 2021 8
SPX 2021 9
SPX 2021 10
SPX 2021 11
SPX 2021 12
SPX 2022 1
SPX 2022 2
SPX 2022 3
SPX 2022 4
SPX 2022 5
SPX 2022 6
SPX 2022 7
SPX 2022 8
SPX 2022 9
SPX 2022 10
SPX 2022 11
SPX 2022 12


In [112]:

import sys
current_dir=r'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/pulling_data'
new_dir=r'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/VIX_SKEW_replication'
os.chdir(new_dir)
from VIX_SKEW_replication.box_client import get_folder_id
get_folder_id()
os.chdir(current_dir)


# CBOE VIX

In [12]:
from CBOE_query import cboe_vix__query
ticks=['vix'] #this is just the S&P500 VIX, CBOE puts other indexes in this file
query=cboe_vix__query(ticks)
#query=cboe_vix__query(['vix'],start='2004-05-08', end='2004-05-12')

df = db.raw_sql(query)
df.dropna(subset=df.columns[1:],inplace=True)
display(df)
dest=fr'CBOE/vix_hist.csv'
df.to_csv(dest)

,date,vix,vixo,vixh,vixl
1011,1990-01-02,17.24,17.24,17.24,17.24
1013,1990-01-03,18.19,18.19,18.19,18.19
1015,1990-01-04,19.22,19.22,19.22,19.22
1017,1990-01-05,20.11,20.11,20.11,20.11
1019,1990-01-08,20.26,20.26,20.26,20.26
...,...,...,...,...,...
13333,2024-10-25,20.33,19.22,20.51,18.23
13334,2024-10-28,19.80,19.11,19.88,18.91
13335,2024-10-29,19.34,19.75,20.53,19.06
13336,2024-10-30,20.35,19.33,20.44,19.30


# Yield Curve

In [29]:
from OptionMetrics_query import yield_curve_query
query=yield_curve_query(start='2022-01-01', end='2023-12-31')
df = db.raw_sql(query)
df.date=pd.to_datetime(df.date)

In [34]:
#import pandas as pd
#df[df.date=='2022-09-26']
df.to_csv(r'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/VIX_SKEW_replication/yc_sample.csv',index=False)
